### **Libraries and Required Logins**

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
from transformers import BitsAndBytesConfig
from torch import cuda
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
print(device)

### **Loading Quantized Models**

In [2]:
# model_id = "meta-llama/Llama-2-7b-chat-hf"
model_id = "mistralai/Mistral-7B-Instruct-v0.1"
hf_auth = ''
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, use_auth_token=hf_auth)
tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=hf_auth)

print(model)
print(f"Model size: {model.get_memory_footprint():,} bytes")

/home2/harshit.g/miniconda3/envs/work/lib/python3.11/site-packages/transformers/models/auto/auto_factory.py:472: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home2/harshit.g/miniconda3/envs/work/lib/python3.11/site-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


/home2/harshit.g/miniconda3/envs/work/lib/python3.11/site-packages/transformers/models/auto/tokenization_auto.py:655: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )

In [3]:
#Using pipeling to activate LangChain
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.bfloat16,
                device_map="auto",
                max_new_tokens = 1024,
                #do_sample=True,
                #top_k=30,
                #num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.eos_token_id,
                )

In [3]:
import json
import textwrap

B_INST, E_INST = "[INST]", "[/INST]"
B_SYS, E_SYS = "<<SYS>>\n", "\n<</SYS>>\n\n"

# DEFAULT_SYSTEM_PROMPT = """\
# You are an evaluation based assistant. You need to rate the summary for the corresponding news with respect to coherency on a scale of 1 to 5, where a rating of 1 means 'incoherent' and 5 means 'perfectly coherent'. You need to provide an explanation for your rating. 
# Note that coherency here means that the summary should be well-structured and well-organized. The summary should not just be a heap of related information, but should build from sentence to sentence to a coherent body of information about a topic.
# First the News shall be provided, following which, the corresponding Summary shall be given.
# """

DEFAULT_SYSTEM_PROMPT = """\
You are a summary evaluation assistant. That means you rate a summary for a news article and provide an explanation for your rating.
"""

"""
DO NOT CHANGE ANY OF THE BELOW FUNCTIONS
"""
def get_prompt(instruction, new_system_prompt=DEFAULT_SYSTEM_PROMPT ):
    SYSTEM_PROMPT = B_SYS + new_system_prompt + E_SYS #system prompt: Default instruction to be given to the model
    prompt_template =  B_INST + SYSTEM_PROMPT + instruction + E_INST # Final Template: takes in instruction as well. Here it would take in the summary and the source
    return prompt_template

#Function to remove the prompt from the final generated answer
def cut_off_text(text, prompt):
    cutoff_phrase = prompt
    index = text.find(cutoff_phrase)
    if index != -1:
        return text[:index]
    else:
        return text

def remove_substring(string, substring):
    return string.replace(substring, "")

### What torch.autocast is: casts tensors to a smaller memory footprint
def generate(text):
    prompt = get_prompt(text)
    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
    outputs = model.generate(**inputs,
                             max_new_tokens=512,
                             eos_token_id=tokenizer.eos_token_id,
                             pad_token_id=tokenizer.eos_token_id,
                             )
    final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
    final_outputs = cut_off_text(final_outputs, '</s>')
    final_outputs = remove_substring(final_outputs, prompt)

    return final_outputs#, outputs

def parse_text(text):
        wrapped_text = textwrap.fill(text, width=100)
        print(wrapped_text +'\n\n')
        # return assistant_text


In [5]:
instruction = """

"""
print("The prompt looks like: \n")
print(get_prompt(instruction))
print("Generated Result is: \n")
print(generate(instruction))

The prompt looks like: 

[INST]<<SYS>>
You are a summary evaluation assistant. That means you rate a summary for a news article and provide an explanation for your rating.

<</SYS>>


Please evaluate the coherence of the summary (enclosed in [SUMMARY] and [/SUMMARY]) for the news article (enclosed in [NEWS] and [/NEWS]) by providing a rating on a scale of 1 to 5, where 1 indicates 'incoherent' and 5 signifies 'perfectly coherent.' 

[NEWS] Neymar helped Brazil beat Chile 1-0 at the Emirates stadium last weekend. Barcelona won the El Clasico to go four points clear at the top of La Liga. Luis Enrique's side take on Celta Vigo in La Liga on Sunday. Neymar and Alves support Barcelona's basketball. Real Madrid is currently atop the Euro League. The two teammates watched basketball together. A recent team in Brazil has won a large contest over their rivals for the year. The player's sister was there to support her brother march to victory. Neyman and teammate Dani Alves showed their support

### LangChain Integration

In [5]:
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain

In [6]:
llm = HuggingFacePipeline(pipeline = pipe, model_kwargs = {'temperature':0})

In [19]:
system_prompt = """You are a summary evaluation assistant. That means you rate a summary for a news and provide an explanation for your rating.
The news article says 'Neymar helped Brazil beat Chile 1-0 at the Emirates stadium last weekend. Barcelona won the El Clasico to go four points clear at the top of La Liga. Luis Enrique's side take on Celta Vigo in La Liga on Sunday. Neymar and Alves support Barcelona's basketball. Real Madrid is currently atop the Euro League. The two teammates watched basketball together. A recent team in Brazil has won a large contest over their rivals for the year. The player's sister was there to support her brother march to victory. Neyman and teammate Dani Alves showed their support to Barcelona by watching the club's basketball game. Barcelona beat Real Madrid in basketball on Thursday night, 85-80. In the stands were Brazil football stars Alves and Neymar. Neymar and Dani Alves, teammates at Barcelona attended an El Clasico of basketball with Rafealle, Neymar's sister. The two player, Neymar and Alves will return from intentional break to play for Barca who currently are top of the La Liga. They are pursuing a treble of trophies this season. Neymar also captained Brazil in their 1-0 victory over Chile at the weekend. Neymar is playing for Barcelona The Brazilian's sister, Rafaella, is who Neymar and Alves watched El Clasico with Neymar helped Brazil to beat Chile Neymar plays for Brazil. Neymar's sister watched the game with Neymar and Alves. Brazil was helped by Neymar to beat Chile at Emirates Stadium. Neymar of Barcelona in the Spanish La Liga attended the basketball El Clasico with his sister, Rafaella and his teammate, Dani Alves. Neymar captained his Brazilian team in their 1-nil victory against Chile played at the Emirates. Barca just beat their rival  at El Clasico 85 to 80. Neymar and his Barcelona teammate Alves went to the club's basketball game along with Neymar's sister Rafaella. Neymar previously helped Brazil beat their rivals on the Chilean team at Emirates Stadium. Barcelona's Nemar and Dani Alves went along with Fafaella to support during a basketball game where Neymar helped brazil win 1-0 over chile.'
"""
instruction = "the summary of the news below. Please evaluate the coherence of the summary for the news by providing a rating on a scale of 1 to 5, where 1 indicates 'incoherent' and 5 signifies 'perfectly coherent'. \n{text}"""
#Please evaluate the coherence of the summary for the news by providing a rating on a scale of 1 to 5, where 1 indicates 'incoherent' and 5 signifies 'perfectly coherent.' 
template = get_prompt(instruction, system_prompt)
print(template)

prompt = PromptTemplate(template=template, input_variables=["text"])
llm_chain = LLMChain(prompt=prompt, llm=llm)""I have given you 

[INST]<<SYS>>
You are a summary evaluation assistant. That means you rate a summary for a news and provide an explanation for your rating.
The news article says 'Neymar helped Brazil beat Chile 1-0 at the Emirates stadium last weekend. Barcelona won the El Clasico to go four points clear at the top of La Liga. Luis Enrique's side take on Celta Vigo in La Liga on Sunday. Neymar and Alves support Barcelona's basketball. Real Madrid is currently atop the Euro League. The two teammates watched basketball together. A recent team in Brazil has won a large contest over their rivals for the year. The player's sister was there to support her brother march to victory. Neyman and teammate Dani Alves showed their support to Barcelona by watching the club's basketball game. Barcelona beat Real Madrid in basketball on Thursday night, 85-80. In the stands were Brazil football stars Alves and Neymar. Neymar and Dani Alves, teammates at Barcelona attended an El Clasico of basketball with Rafealle, Neym

In [21]:
text = """ 
The summary of the above news article is 'team-mates neymar and dani alves proved their dedication to barcelona by supporting the club's basketball side.' """
output = llm_chain.run(text)
parse_text(output)

  Thank you for providing the news article and the summary for evaluation. Based on the coherence
and accuracy of the summary, I would rate it a 4 out of 5. The summary provides a clear and concise
overview of the news article, highlighting the main points and key information. It accurately
conveys the message of the article, including the involvement of Neymar and Dani Alves in supporting
Barcelona's basketball team, their previous victory over Chile, and their current pursuit of a
treble of trophies. The summary also uses appropriate language and sentence structure to convey the
information effectively. However, there is a minor flaw in the summary. The phrase "team-mates
neymar and dani alves proved their dedication to barcelona by supporting the club's basketball side"
is a bit simplistic and doesn't fully capture the context and nuances of the original article. For
example, the article mentions that Neymar captained Brazil in their victory over Chile, which is not
explicitly menti

### Conversational Memory

In [7]:
from langchain import HuggingFacePipeline
from langchain import PromptTemplate,  LLMChain
from langchain.memory import ConversationBufferMemory
from langchain import LLMChain, PromptTemplate

In [8]:
instruction = "Chat History:\n\n{chat_history} \n\nUser: {user_input}"
system_prompt = "You are a summary evaluation assistant. That means you rate a summary for a news article and provide an explanation for your rating. You always only answer for the assistant then you stop. read the chat history to get context"

template = get_prompt(instruction, system_prompt)
print(template)


[INST]<<SYS>>
You are a summary evaluation assistant. That means you rate a summary for a news article and provide an explanation for your rating. You always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

{chat_history} 

User: {user_input}[/INST]


In [14]:
prompt = PromptTemplate(
    input_variables=["chat_history", "user_input"], template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory,
)

In [15]:
#You need to rate the summary for the corresponding news with respect to coherency on a scale of 1 to 5, where a rating of 1 means 'incoherent' and 5 means 'perfectly coherent'. You need to provide an explanation for your rating. 
# Note that coherency here means that the summary should be well-structured and well-organized. The summary should not just be a heap of related information, but should build from sentence to sentence to a coherent body of information about a topic.
llm_chain.predict(user_input="""I have written a summary for a news article. I need you to rate the summary for the corresponding news with respect to coherency on a scale of 1 to 5, where a rating of 1 means 'incoherent' and 5 means 'perfectly coherent'. You need to provide an explanation for your rating. 
Please note that Note that coherency here means that the summary should be well-structured and well-organized. The summary should not just be a heap of related information, but should build from sentence to sentence to a coherent body of information about a topic. 
What should I provide and in which format, so that you can give me the rating?
""")

###I wrote a summary of a news article. The summary is: 'Harshit is a 9 year old boy who dances well'. 
#The news article which I summarized is: 'Neymar helped Brazil beat Chile 1-0 at the Emirates stadium last weekend. Barcelona won the El Clasico to go four points clear at the top of La Liga. Luis Enrique's side take on Celta Vigo in La Liga on Sunday. Neymar and Alves support Barcelona's basketball. Real Madrid is currently atop the Euro League. The two teammates watched basketball together. A recent team in Brazil has won a large contest over their rivals for the year. The player's sister was there to support her brother march to victory. Neyman and teammate Dani Alves showed their support to Barcelona by watching the club's basketball game. Barcelona beat Real Madrid in basketball on Thursday night, 85-80. In the stands were Brazil football stars Alves and Neymar. Neymar and Dani Alves, teammates at Barcelona attended an El Clasico of basketball with Rafealle, Neymar's sister. The two player, Neymar and Alves will return from intentional break to play for Barca who currently are top of the La Liga. They are pursuing a treble of trophies this season. Neymar also captained Brazil in their 1-0 victory over Chile at the weekend. Neymar is playing for Barcelona The Brazilian's sister, Rafaella, is who Neymar and Alves watched El Clasico with Neymar helped Brazil to beat Chile Neymar plays for Brazil. Neymar's sister watched the game with Neymar and Alves. Brazil was helped by Neymar to beat Chile at Emirates Stadium. Neymar of Barcelona in the Spanish La Liga attended the basketball El Clasico with his sister, Rafaella and his teammate, Dani Alves. Neymar captained his Brazilian team in their 1-nil victory against Chile played at the Emirates. Barca just beat their rival  at El Clasico 85 to 80. Neymar and his Barcelona teammate Alves went to the club's basketball game along with Neymar's sister Rafaella. Neymar previously helped Brazil beat their rivals on the Chilean team at Emirates Stadium. Barcelona's Nemar and Dani Alves went along with Fafaella to support during a basketball game where Neymar helped brazil win 1-0 over chile.'. 
#I need your help to evaluate the coherence of the summary (given in the beginning) by providing a rating on a scale of 1 to 5, where 1 indicates 'incoherent' and 5 signifies 'perfectly coherent.' Please note that Note that coherency here means that the summary should be well-structured and well-organized. The summary should not just be a heap of related information, but should build from sentence to sentence to a coherent body of information about a topic.




> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a summary evaluation assistant. That means you rate a summary for a news article and provide an explanation for your rating. You always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

 

User: I have written a summary for a news article. I need you to rate the summary for the corresponding news with respect to coherency on a scale of 1 to 5, where a rating of 1 means 'incoherent' and 5 means 'perfectly coherent'. You need to provide an explanation for your rating. 
Please note that Note that coherency here means that the summary should be well-structured and well-organized. The summary should not just be a heap of related information, but should build from sentence to sentence to a coherent body of information about a topic. 
What should I provide and in which format, so that you can give me the rating?
[/INST]

> Finished chain.


"  Great, I'm ready to help! To rate the summary for coherency, I will need you to provide the summary of the news article in a clear and concise format. Please paste the summary into the chat window, and I will evaluate it based on the criteria you provided.\nOnce I have the summary, I will give it a rating on a scale of 1 to 5, where 1 means incoherent and 5 means perfectly coherent. I will also provide an explanation for my rating, highlighting the strengths and weaknesses of the summary.\nPlease go ahead and provide the summary, and I will get started on the evaluation."

In [16]:
llm_chain.predict(user_input="""
The summary is: paul merson has restarted his row with andros townsend . the tottenham midfielder was brought on with only seven minutes remaining in his team 's 0-0 draw with burnley . andros townsend scores england 's equaliser in their 1-1 friendly draw with italy in turin .

And the news article for which I wrote the above summary is: Andros Townsend an 83rd minute sub in Tottenham's draw with Burnley. He was unable to find a winner as the game ended without a goal. Townsend had clashed with Paul Merson last week over England call-up. Sports columnist Paul Merson and Andros Townsend are in the midst of a twitter feud. Merson started it when Townsend was called up and wrote something disparaging about him in his column. Since then things have gone back and forth between the two. Merson is angered by the statement made by Townsend in his Sky Sports column. Merson threw a dig at Townsend after scoring his last game. Paul Merson has officially restarted his row line up with Andros Townsend. This is a result of seeing Townsend's performance in the last seven minutes of the game the previous Sunday. An athlete was brought in to save the game during an event against a rival team.  Although many disagreed with this decision as players have been known to get in trouble from time to time. Paul Merson and Andros Townsend have been in a bout for a while now, Merson felt that Townsend did not deserve a place in the English team. Townsend scored for England with a crucial goal to which Merson apologized and acknowledge the performance of Townsend in that game and wished him well on his performance. The back and forth between the two men has continued regardless but it appears that now their bad feelings have subsided despite some light jest between the two. Paul Merson restarted his fued with the former Arenal player Andros Townsend. Townsend the former Arsenal player scored a goal for England against italy. Townsend replied to Mersons remarks on Twitter by saying "not bad for a player that should be` nowhere near the squad 'ay" Paul Merson used to play for Arsenal. Andros Townsend has been criticized by Paul Merson. The goal scored against Italy was made by Townsend. Paul Merson, former Arsenal player and Andros Townsend have restarted their bout of banter. Merson tweeted about Townsend and his poor performance making it clear he did not believe that Townsend was good enough to play for England. Townsend however scored the crucial goal for England vs Italy. Merson swallowed his criticism of Townsend and acknowledge his performance. The two are in a good place now however and any subsequent banter between the two appears to have been in good taste. Andros Townsend used to play for arsenal. Sky sports pundit Merson criticized Andros Townsend. Andros Townsend scored a goal against Italy to a draw. Paul Merson, the Sky Sports pundit, criticized Andros Townsend last week after his call-up to the england squad.  Merson admitted it was a mistake after Townsend scored, bringing the match against Italy to a tie on Tuesday.  Merson is a former Arsenal player himself.

Also, just curious, but what do you think about football?
""")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a summary evaluation assistant. That means you rate a summary for a news article and provide an explanation for your rating. You always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

Human: I have written a summary for a news article. I need you to rate the summary for the corresponding news with respect to coherency on a scale of 1 to 5, where a rating of 1 means 'incoherent' and 5 means 'perfectly coherent'. You need to provide an explanation for your rating. 
Please note that Note that coherency here means that the summary should be well-structured and well-organized. The summary should not just be a heap of related information, but should build from sentence to sentence to a coherent body of information about a topic. 
What should I provide and in which format, so that you can give me the rating?

AI:   Great, I'm ready to help! To rate the s

'  Based on the summary provided, I would rate it a 4 out of 5 for coherency. The summary provides a clear and concise overview of the news article, including the main points and key details. The structure of the summary is well-organized, with each sentence building on the previous one to create a coherent body of information.\nThe only area for improvement is the use of transitions between sentences. At times, the summary jumps abruptly from one sentence to the next, which can make it difficult to follow the flow of ideas. Adding transitional phrases or sentences to connect the ideas more smoothly would enhance the overall coherency of the summary.\nOverall, a great job!'

In [17]:
llm_chain.predict(user_input="Just to make sure, can you print what summary I have provided to you and what news article I provided to you?")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a summary evaluation assistant. That means you rate a summary for a news article and provide an explanation for your rating. You always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

Human: I have written a summary for a news article. I need you to rate the summary for the corresponding news with respect to coherency on a scale of 1 to 5, where a rating of 1 means 'incoherent' and 5 means 'perfectly coherent'. You need to provide an explanation for your rating. 
Please note that Note that coherency here means that the summary should be well-structured and well-organized. The summary should not just be a heap of related information, but should build from sentence to sentence to a coherent body of information about a topic. 
What should I provide and in which format, so that you can give me the rating?

AI:   Great, I'm ready to help! To rate the s

'  Of course! Here is the summary you provided:\n"Paul Merson has restarted his row with Andros Townsend. The Tottenham midfielder was brought on with only seven minutes remaining in his team\'s 0-0 draw with Burnley. Andros Townsend scores England\'s equaliser in their 1-1 friendly draw with Italy in Turin."\nAnd here is the news article you provided as context:\n"Andros Townsend an 83rd minute sub in Tottenham\'s draw with Burnley. He was unable to find a winner as the game ended without a goal. Townsend had clashed with Paul Merson last week over England call-up."\nPlease let me know if you have any other questions!'

In [13]:
llm_chain.predict(user_input="How are you doing? what is the weather like today?")



> Entering new LLMChain chain...
Prompt after formatting:
[INST]<<SYS>>
You are a summary evaluation assistant. That means you rate a summary for a news article and provide an explanation for your rating. You always only answer for the assistant then you stop. read the chat history to get context
<</SYS>>

Chat History:

Human: I have written a summary for a news article. I need you to rate the summary for the corresponding news with respect to coherency on a scale of 1 to 5, where a rating of 1 means 'incoherent' and 5 means 'perfectly coherent'. You need to provide an explanation for your rating. 
Please note that Note that coherency here means that the summary should be well-structured and well-organized. The summary should not just be a heap of related information, but should build from sentence to sentence to a coherent body of information about a topic. 
What should I provide and in which format, so that you can give me the rating?

AI:   Hello! As a summary evaluation assistan

"  Hello! *adjusts glasses* As a summary evaluation assistant, I don't have access to real-time weather information. However, I can tell you that the weather is always coherent and organized, just like a well-written summary! *winks*"

## Extra Old Stuff

In [26]:
text = """
### Instruction:
Rate the summary for the corresponding news with respect to coherency on a scale of 1 to 5, where a rating of 1 means 'incoherent' and 5 means 'perfectly coherent'. Provide an explanation for your rating. Note that coherency here means that the summary should be well-structured and well-organized. The summary should not just be a heap of related information, but should build from sentence to sentence to a coherent body of information about a topic.

News: Neymar helped Brazil beat Chile 1-0 at the Emirates stadium last weekend. Barcelona won the El Clasico to go four points clear at the top of La Liga. Luis Enrique's side take on Celta Vigo in La Liga on Sunday. Neymar and Alves support Barcelona's basketball. Real Madrid is currently atop the Euro League. The two teammates watched basketball together. A recent team in Brazil has won a large contest over their rivals for the year. The player's sister was there to support her brother march to victory. Neyman and teammate Dani Alves showed their support to Barcelona by watching the club's basketball game. Barcelona beat Real Madrid in basketball on Thursday night, 85-80. In the stands were Brazil football stars Alves and Neymar. Neymar and Dani Alves, teammates at Barcelona attended an El Clasico of basketball with Rafealle, Neymar's sister. The two player, Neymar and Alves will return from intentional break to play for Barca who currently are top of the La Liga. They are pursuing a treble of trophies this season. Neymar also captained Brazil in their 1-0 victory over Chile at the weekend. Neymar is playing for Barcelona The Brazilian's sister, Rafaella, is who Neymar and Alves watched El Clasico with Neymar helped Brazil to beat Chile Neymar plays for Brazil. Neymar's sister watched the game with Neymar and Alves. Brazil was helped by Neymar to beat Chile at Emirates Stadium. Neymar of Barcelona in the Spanish La Liga attended the basketball El Clasico with his sister, Rafaella and his teammate, Dani Alves. Neymar captained his Brazilian team in their 1-nil victory against Chile played at the Emirates. Barca just beat their rival  at El Clasico 85 to 80. Neymar and his Barcelona teammate Alves went to the club's basketball game along with Neymar's sister Rafaella. Neymar previously helped Brazil beat their rivals on the Chilean team at Emirates Stadium. Barcelona's Nemar and Dani Alves went along with Fafaella to support during a basketball game where Neymar helped brazil win 1-0 over chile.

Summary: Neymar Emirates stadium beat 1-0 last weekend.
"""
#Summary: team-mates neymar and dani alves proved their dedication to barcelona by supporting the club 's basketball side . neymar and alves headed to watch el clasico on thursday night alongside the brazilian 's sister rafaella . barca prevailed with a narrow 85-80 victory in the euro league contest .
#Summary: Neymar Emirates stadium beat 1-0 last weekend.
inputs = tokenizer(text, return_tensors="pt").to(device)
# generation_config = GenerationConfig(
#         temperature=0.1,
#         top_p=0.75,
#         repetition_penalty=1.1,
# )
with torch.inference_mode():
    outputs = model.generate(
        **inputs,
        # generation_config=generation_config,
        max_new_tokens=400,
        )
# outputs = model.generate(**inputs, max_new_tokens=200)

In [27]:
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


### Instruction:
Rate the summary for the corresponding news with respect to coherency on a scale of 1 to 5, where a rating of 1 means 'incoherent' and 5 means 'perfectly coherent'. Provide an explanation for your rating. Note that coherency here means that the summary should be well-structured and well-organized. The summary should not just be a heap of related information, but should build from sentence to sentence to a coherent body of information about a topic. Give output in format of Rating: and Explanation:

News: Neymar helped Brazil beat Chile 1-0 at the Emirates stadium last weekend. Barcelona won the El Clasico to go four points clear at the top of La Liga. Luis Enrique's side take on Celta Vigo in La Liga on Sunday. Neymar and Alves support Barcelona's basketball. Real Madrid is currently atop the Euro League. The two teammates watched basketball together. A recent team in Brazil has won a large contest over their rivals for the year. The player's sister was there to suppo

In [28]:
text = """
### Instruction:
Rate the summary for the corresponding news with respect to consistency on a scale of 1 to 5, where a rating of 1 means 'inconsistent' and 5 means 'perfectly consistent'. Provide an explanation for your rating. Note that consistency here means that the summary contains only statements that are entailed by the news article. Consistency also involves peanalizing summaries that contained hallucinated facts. 

News: Neymar helped Brazil beat Chile 1-0 at the Emirates stadium last weekend. Barcelona won the El Clasico to go four points clear at the top of La Liga. Luis Enrique's side take on Celta Vigo in La Liga on Sunday. Neymar and Alves support Barcelona's basketball. Real Madrid is currently atop the Euro League. The two teammates watched basketball together. A recent team in Brazil has won a large contest over their rivals for the year. The player's sister was there to support her brother march to victory. Neyman and teammate Dani Alves showed their support to Barcelona by watching the club's basketball game. Barcelona beat Real Madrid in basketball on Thursday night, 85-80. In the stands were Brazil football stars Alves and Neymar. Neymar and Dani Alves, teammates at Barcelona attended an El Clasico of basketball with Rafealle, Neymar's sister. The two player, Neymar and Alves will return from intentional break to play for Barca who currently are top of the La Liga. They are pursuing a treble of trophies this season. Neymar also captained Brazil in their 1-0 victory over Chile at the weekend. Neymar is playing for Barcelona The Brazilian's sister, Rafaella, is who Neymar and Alves watched El Clasico with Neymar helped Brazil to beat Chile Neymar plays for Brazil. Neymar's sister watched the game with Neymar and Alves. Brazil was helped by Neymar to beat Chile at Emirates Stadium. Neymar of Barcelona in the Spanish La Liga attended the basketball El Clasico with his sister, Rafaella and his teammate, Dani Alves. Neymar captained his Brazilian team in their 1-nil victory against Chile played at the Emirates. Barca just beat their rival  at El Clasico 85 to 80. Neymar and his Barcelona teammate Alves went to the club's basketball game along with Neymar's sister Rafaella. Neymar previously helped Brazil beat their rivals on the Chilean team at Emirates Stadium. Barcelona's Nemar and Dani Alves went along with Fafaella to support during a basketball game where Neymar helped brazil win 1-0 over chile.

Summary: team-mates neymar and dani alves proved their dedication to barcelona by supporting the club 's basketball side . neymar and alves headed to watch el clasico on thursday night alongside the brazilian 's sister rafaella . barca prevailed with a narrow 85-80 victory in the euro league contest .
"""
#Give output in format of Rating: and Explanation:
#Summary: team-mates neymar and dani alves proved their dedication to barcelona by supporting the club 's basketball side . neymar and alves headed to watch el clasico on thursday night alongside the brazilian 's sister rafaella . barca prevailed with a narrow 85-80 victory in the euro league contest .
#Summary: Neymar Emirates stadium beat 1-0 last weekend.
inputs = tokenizer(text, return_tensors="pt").to(device)
# generation_config = GenerationConfig(
#         temperature=0.1,
#         top_p=0.75,
#         repetition_penalty=1.1,
# )
with torch.inference_mode():
    outputs = model.generate(
        **inputs,
        # generation_config=generation_config,
        max_new_tokens=400,
        )
# outputs = model.generate(**inputs, max_new_tokens=200)